In [12]:
#导入库
import pandas as pd
import numpy as np
from pyecharts.charts import Bar, Grid, Line, Pie
import pyecharts.options as opts
import datetime 
import warnings
warnings.filterwarnings("ignore")

In [13]:
#导入数据
databuy=pd.read_csv('E:数据分析师相关/数据分析项目相关/淘宝母婴购物行为数据集/tianchi_mum_baby_trade_history.csv')

In [14]:
#数据查看
databuy.info()
print(databuy.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29971 entries, 0 to 29970
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   user_id     29971 non-null  int64
 1   auction_id  29971 non-null  int64
 2   category_2  29971 non-null  int64
 3   category_1  29971 non-null  int64
 4   buy_mount   29971 non-null  int64
 5   day         29971 non-null  int64
dtypes: int64(6)
memory usage: 1.4 MB
user_id       0
auction_id    0
category_2    0
category_1    0
buy_mount     0
day           0
dtype: int64


In [15]:
databuy.head(5)

,user_id,auction_id,category_2,category_1,buy_mount,day
0,786295544,41098319944,50014866,50022520,2,20140919
1,532110457,17916191097,50011993,28,1,20131011
2,249013725,21896936223,50012461,50014815,1,20131011
3,917056007,12515996043,50018831,50014815,2,20141023
4,444069173,20487688075,50013636,50008168,1,20141103


In [16]:
databuy.duplicated().sum()

0

In [17]:
databuy.buy_mount.describe()

count    29971.000000
mean         2.544126
std         63.986879
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max      10000.000000
Name: buy_mount, dtype: float64

In [18]:
#数据异常值处理（购买量最大为10000，不合理）
yc=(databuy['buy_mount'].mean())+3*(databuy['buy_mount'].std())
databuy=databuy[databuy['buy_mount']<=int(yc)]

In [19]:
#数据处理
#类型
databuy['user_id']=databuy.user_id.astype(str)
databuy['auction_id']=databuy.auction_id.astype(str)
databuy['category_1']=databuy.category_1.astype(str)
databuy['category_2']=databuy.category_2.astype(str)
databuy['day']=databuy['day'].astype(str)
databuy['day']=pd.to_datetime(databuy['day'])

In [20]:
databuy=databuy.sort_values(by='day').reset_index(drop=True)

In [21]:
print("淘宝母婴购物订单量：",databuy['user_id'].count())
print("用户数：",databuy['user_id'].nunique())
#print("交易数：",databuy['auction_id'].nunique())
print("商品一级类目数：",databuy['category_1'].nunique())
print("商品二级类目数：",databuy['category_2'].nunique())
print("商品购买总量：",databuy['buy_mount'].sum())
print("平均每用户商品购买量：",round(databuy['buy_mount'].sum()/databuy['user_id'].nunique(),2))
print("平均每订单购买量：",round(databuy['buy_mount'].sum()/databuy['user_id'].count(),2))
print("订单日期：",databuy['day'].min().date(),"至",databuy['day'].max().date())

淘宝母婴购物订单量： 29941
用户数： 29914
商品一级类目数： 6
商品二级类目数： 662
商品购买总量： 49778
平均每用户商品购买量： 1.66
平均每订单购买量： 1.66
订单日期： 2012-07-02 至 2015-02-05


In [22]:
#字段
databuy['year']=databuy['day'].dt.year.astype(str)
databuy['quarter']=databuy['day'].dt.quarter.astype(str)
databuy['month']=databuy['day'].dt.strftime('%m')
databuy['days']=databuy['day'].dt.strftime('%d')

In [23]:
databuy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29941 entries, 0 to 29940
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     29941 non-null  object        
 1   auction_id  29941 non-null  object        
 2   category_2  29941 non-null  object        
 3   category_1  29941 non-null  object        
 4   buy_mount   29941 non-null  int64         
 5   day         29941 non-null  datetime64[ns]
 6   year        29941 non-null  object        
 7   quarter     29941 non-null  object        
 8   month       29941 non-null  object        
 9   days        29941 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 2.3+ MB


In [24]:
databuy.describe()

,buy_mount
count,29941.000000
mean,1.662536
std,5.145759
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,192.000000


In [25]:
databuy.head(5)

,user_id,auction_id,category_2,category_1,buy_mount,day,year,quarter,month,days
0,901458847,9957236312,50023792,28,1,2012-07-02,2012,3,07,02
1,90557762,14792803943,50016032,28,1,2012-07-02,2012,3,07,02
2,633375258,14794202119,50013207,50008168,2,2012-07-02,2012,3,07,02
3,715517110,14804987693,50012437,50014815,1,2012-07-02,2012,3,07,02
4,88516466,14673891903,50014260,28,1,2012-07-02,2012,3,07,02


In [26]:
#流量分析
#年销量分析
sales_y=databuy.groupby('year')['buy_mount'].sum().reset_index().sort_values(by='year')

In [27]:
sy=Line()
sy.add_xaxis(xaxis_data=list(sales_y.year))
sy.add_yaxis(
    series_name="年销量",
    y_axis=list(sales_y.buy_mount),
    label_opts=opts.LabelOpts(is_show=False),
    markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='min'),opts.MarkPointItem(type_='max')])
)
sy.set_global_opts(
    tooltip_opts=opts.TooltipOpts(is_show=True,trigger='axis',axis_pointer_type='shadow'),
    xaxis_opts=opts.AxisOpts(type_='category'),
    yaxis_opts=opts.AxisOpts(type_='value'),
    title_opts=opts.TitleOpts(title="每年的商品销量情况")
)
sy.render_notebook()

所分析的数据是包含2012-7-2到2015-2-5的淘宝母婴购物信息。主要观察2013、2014年数据。
由图可以看出，2013-2014年淘宝母婴购物销量呈增长形势。
在大环境不变的情况下，预计2015年淘宝母婴购物销量也会增加。

In [28]:
#季度销量分析
sales_jd=databuy.groupby(['year','quarter'])['buy_mount'].sum().reset_index().sort_values(by=['year','quarter'])
sales_jd['jidu']=sales_jd['year']+'/Q'+sales_jd['quarter']

In [29]:
sq=Line()
sq.add_xaxis(xaxis_data=list(sales_jd.jidu))
sq.add_yaxis(
    series_name="季度销量",
    y_axis=list(sales_jd.buy_mount),
    label_opts=opts.LabelOpts(is_show=False),
    markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='min'),opts.MarkPointItem(type_='max')])
)
sq.set_global_opts(
    tooltip_opts=opts.TooltipOpts(is_show=True,trigger='axis',axis_pointer_type='shadow'),
    xaxis_opts=opts.AxisOpts(type_='category'),
    yaxis_opts=opts.AxisOpts(type_='value'),
    title_opts=opts.TitleOpts(title="每季度的商品销量情况")
)
sq.render_notebook()

由图可以看出，每年的第一季度母婴商品销量最少，第四季度销量最多。
可能是因为第四季度有双十一活动，第一季度促销活动较少、活动力度不大。后面再进一步分析。

In [30]:
#月销量分析
sales_ny=databuy.groupby(['year','month'])['buy_mount'].sum().reset_index().sort_values(by=['year','month'])
sales_ny['nyue']=sales_ny['year']+'-'+sales_ny['month']

In [31]:
sm=Line()
sm.add_xaxis(xaxis_data=list(sales_ny.nyue))
sm.add_yaxis(
    series_name="月销量",
    y_axis=list(sales_ny.buy_mount),
    label_opts=opts.LabelOpts(is_show=False),
    markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='min'),opts.MarkPointItem(type_='max')])
)
sm.set_global_opts(
    tooltip_opts=opts.TooltipOpts(is_show=True,trigger='axis',axis_pointer_type='shadow'),
    xaxis_opts=opts.AxisOpts(type_='category'),
    yaxis_opts=opts.AxisOpts(type_='value'),
    title_opts=opts.TitleOpts(title="每月的商品销量情况")
)
sm.render_notebook()

由图可以看出，每年的5月和11月是母婴商品销量的高峰，
而2013年2月、2014年1月、2014年6月是母婴商品销量的低峰。接下来对这些月份的数据进一步分析。

In [34]:
#日销量分析
#2013、2014年5月
sales_1405=databuy[(databuy.year=="2014")&(databuy.month=="05")].groupby('days')['buy_mount'].sum().reset_index()
sales_1305=databuy[(databuy.year=="2013")&(databuy.month=="05")].groupby('days')['buy_mount'].sum().reset_index()

s05=Line()
s05.add_xaxis(xaxis_data=list(sales_1405.days))
s05.add_yaxis(
    series_name="14年日销量",
    y_axis=list(sales_1405.buy_mount),
    label_opts=opts.LabelOpts(is_show=False),
    markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='min'),opts.MarkPointItem(type_='max')]),
    markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average')])
)
s05.add_yaxis(
    series_name="13年日销量",
    y_axis=list(sales_1305.buy_mount),
    label_opts=opts.LabelOpts(is_show=False),
    markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='min'),opts.MarkPointItem(type_='max')]),
    markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average')])
)
s05.set_global_opts(
    tooltip_opts=opts.TooltipOpts(is_show=True,trigger='axis',axis_pointer_type='shadow'),
    xaxis_opts=opts.AxisOpts(type_='category'),
    yaxis_opts=opts.AxisOpts(type_='value'),
    title_opts=opts.TitleOpts(title="2013、2014年5月的商品销量情况")
)
s05.render_notebook()

由图可以看出，2013、2014年5月12、13日（母亲节附近）和20日（520）母婴商品销量较高，此外不定期销量突增。
5月份销量高可能是因为母亲节活动、520活动，以及其他的不定期品牌促销活动。

In [23]:
#2014年11月
sales_1411=databuy[(databuy.year=="2014")&(databuy.month=="11")].groupby('days')['buy_mount'].sum().reset_index()

s1411=Line()
s1411.add_xaxis(xaxis_data=list(sales_1411.days))
s1411.add_yaxis(
    series_name="日销量",
    y_axis=list(sales_1411.buy_mount),
    label_opts=opts.LabelOpts(is_show=False),
    markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='min'),opts.MarkPointItem(type_='max')]),
    markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average')])
)
s1411.set_global_opts(
    tooltip_opts=opts.TooltipOpts(is_show=True,trigger='axis',axis_pointer_type='shadow'),
    xaxis_opts=opts.AxisOpts(type_='category'),
    yaxis_opts=opts.AxisOpts(type_='value'),
    title_opts=opts.TitleOpts(title="2014年11月的商品销量情况")
)
s1411.render_notebook()

由图可以看出，11月份销量高主要是因为双十一活动。

In [24]:
#2014年1月
sales_1401=databuy[(databuy.year=="2014")&(databuy.month=="01")].groupby('days')['buy_mount'].sum().reset_index()

s1401=Line()
s1401.add_xaxis(xaxis_data=list(sales_1401.days))
s1401.add_yaxis(
    series_name="日销量",
    y_axis=list(sales_1401.buy_mount),
    label_opts=opts.LabelOpts(is_show=False),
    markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='min'),opts.MarkPointItem(type_='max')]),
    markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average')])
)
s1401.set_global_opts(
    tooltip_opts=opts.TooltipOpts(is_show=True,trigger='axis',axis_pointer_type='shadow'),
    xaxis_opts=opts.AxisOpts(type_='category'),
    yaxis_opts=opts.AxisOpts(type_='value'),
    title_opts=opts.TitleOpts(title="2014年1月的商品销量情况")
)
s1401.render_notebook()

In [25]:
#2013年2月
sales_1302=databuy[(databuy.year=="2013")&(databuy.month=="02")].groupby('days')['buy_mount'].sum().reset_index()

s1302=Line()
s1302.add_xaxis(xaxis_data=list(sales_1302.days))
s1302.add_yaxis(
    series_name="日销量",
    y_axis=list(sales_1302.buy_mount),
    label_opts=opts.LabelOpts(is_show=False),
    markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='min'),opts.MarkPointItem(type_='max')]),
    markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average')])
)
s1302.set_global_opts(
    tooltip_opts=opts.TooltipOpts(is_show=True,trigger='axis',axis_pointer_type='shadow'),
    xaxis_opts=opts.AxisOpts(type_='category'),
    yaxis_opts=opts.AxisOpts(type_='value'),
    title_opts=opts.TitleOpts(title="2013年2月的商品销量情况")
)
s1302.render_notebook()

因为2013年2月10日和2014年1月31日是春节，由图可以看出，春节前一个月和后一周母婴商品销量都很低，
因此，2013年2月和2014年1月销量低是因为处于春节期间，可能没有进行促销活动。可以在春节前后增加促销活动，刺激消费。

In [26]:
#2014年6月
sales_1406=databuy[(databuy.year=="2014")&(databuy.month=="06")].groupby('days')['buy_mount'].sum().reset_index()

s1406=Line()
s1406.add_xaxis(xaxis_data=list(sales_1406.days))
s1406.add_yaxis(
    series_name="日销量",
    y_axis=list(sales_1406.buy_mount),
    label_opts=opts.LabelOpts(is_show=False),
    markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='min'),opts.MarkPointItem(type_='max')]),
    markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average')])
)
s1406.set_global_opts(
    tooltip_opts=opts.TooltipOpts(is_show=True,trigger='axis',axis_pointer_type='shadow'),
    xaxis_opts=opts.AxisOpts(type_='category'),
    yaxis_opts=opts.AxisOpts(type_='value'),
    title_opts=opts.TitleOpts(title="2014年6月的商品销量情况")
)
s1406.render_notebook()

由图可以看出，2014年6月份母婴商品销量较为均衡，618活动期间销量有所增加，但增加不多。
因此，2014年6月母婴商品销量较少可能是因为618活动力度不大，也可能是因为5月活动力度大、消费多，使得6月需求减少。

In [35]:
#2014年12月
sales_1412=databuy[(databuy.year=="2014")&(databuy.month=="12")].groupby('days')['buy_mount'].sum().reset_index()

s1412=Line()
s1412.add_xaxis(xaxis_data=list(sales_1412.days))
s1412.add_yaxis(
    series_name="日销量",
    y_axis=list(sales_1412.buy_mount),
    label_opts=opts.LabelOpts(is_show=False),
    markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='min'),opts.MarkPointItem(type_='max')]),
    markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average')])
)
s1412.set_global_opts(
    tooltip_opts=opts.TooltipOpts(is_show=True,trigger='axis',axis_pointer_type='shadow'),
    xaxis_opts=opts.AxisOpts(type_='category'),
    yaxis_opts=opts.AxisOpts(type_='value'),
    title_opts=opts.TitleOpts(title="2014年12月的商品销量情况")
)
s1412.render_notebook()

In [27]:
#类别分析
print("每年商品一级类目数：\n",databuy.groupby('year')['category_1'].nunique())
print("每年商品二级类目数：\n",databuy.groupby('year')['category_2'].nunique())

每年商品一级类目数：
 year
2012    6
2013    6
2014    6
2015    6
Name: category_1, dtype: int64
每年商品二级类目数：
 year
2012    396
2013    444
2014    576
2015    294
Name: category_2, dtype: int64


In [28]:
#2014年
#贡献前80%母婴购物销量的商品一级类目分析
buy14=databuy[databuy['year']=='2014']
print("2014年母婴商品一级类目数：",buy14['category_1'].nunique())
sales_lei1=buy14.groupby('category_1')['buy_mount'].sum().sort_values(ascending=False).reset_index()

sales_14z=sales_lei1['buy_mount'].sum()
sales_lei1z=sales_lei1['category_1'].count()

sales_lei1['cumsum']=sales_lei1['buy_mount'].cumsum()
sales_lei1['cs_percent']=round(sales_lei1['cumsum']/sales_14z,2)
sales_lei1m80=sales_lei1[sales_lei1['cs_percent']<=0.8]
sales_lei180n=sales_lei1m80['category_1'].count()
print("贡献前80%母婴购物销量的商品一级类目数：",sales_lei180n)
sales_lei180=round(sales_lei180n/sales_lei1z,2)
print("贡献前80%母婴购物销量的商品一级类目数占比：",sales_lei180)

2014年母婴商品一级类目数： 6
贡献前80%母婴购物销量的商品一级类目数： 2
贡献前80%母婴购物销量的商品一级类目数占比： 0.33


In [29]:
print(list(sales_lei1.category_1))
sl1_bar=Bar()
sl1_bar.add_xaxis(xaxis_data=list(sales_lei1.category_1))
sl1_bar.add_yaxis(
    series_name="销量",
    y_axis=list(sales_lei1.buy_mount),
    yaxis_index=0
)
sl1_bar.add_yaxis(
    series_name="前80%销量",
    y_axis=list(sales_lei1m80.buy_mount),
    yaxis_index=0
)
sl1_bar.extend_axis(
    yaxis=opts.AxisOpts(
        type_="value",
        name="销量占比",
        min_=0,
        max_=1,
        position="right",
    )
)
sl1_bar.set_global_opts(
    tooltip_opts=opts.TooltipOpts(is_show=True,trigger='axis',axis_pointer_type='cross'),
    xaxis_opts=opts.AxisOpts(type_='category'),
    yaxis_opts=opts.AxisOpts(type_='value',min_=0, max_=10000, position="left"),
    title_opts=opts.TitleOpts(title="一级类目商品销量以及贡献前80%销量的情况"),
    legend_opts=opts.LegendOpts(pos_top="18")
)

sl1_line=Line()
sl1_line.add_xaxis(xaxis_data=list(sales_lei1.category_1))
sl1_line.add_yaxis(
    series_name="销量占比",
    y_axis=list(sales_lei1.cs_percent),
    yaxis_index=1
)

sl1_bar.overlap(sl1_line)
grid=Grid()
grid.add(
    sl1_bar, 
    opts.GridOpts(pos_left="5%", pos_right="5%"),
    is_control_axis_index=True
)
grid.render_notebook()

['28', '50008168', '50014815', '38', '50022520', '122650008']


In [30]:
#2014年
#贡献前80%母婴购物销量的商品二级类目分析
print("2014年母婴商品二级类目数：",buy14['category_2'].nunique())
sales_lei2=buy14.groupby('category_2')['buy_mount'].sum().sort_values(ascending=False).reset_index()

sales_lei2z=sales_lei2['category_2'].count()

sales_lei2['cumsum']=sales_lei2['buy_mount'].cumsum()
sales_lei2['cs_percent']=round(sales_lei2['cumsum']/sales_14z,2)
sales_lei2m80=sales_lei2[sales_lei2['cs_percent']<=0.8]
sales_lei280n=sales_lei2m80['category_2'].count()
print("贡献前80%母婴购物销量的商品二级类目数：",sales_lei280n)
sales_lei280=round(sales_lei280n/sales_lei2z,2)
print("贡献前80%母婴购物销量的商品二级类目数占比：",sales_lei280)

2014年母婴商品二级类目数： 576
贡献前80%母婴购物销量的商品二级类目数： 116
贡献前80%母婴购物销量的商品二级类目数占比： 0.2


In [31]:
print(list(sales_lei2m80.category_2))
sl2_bar=Bar()
sl2_bar.add_xaxis(xaxis_data=list(sales_lei2m80.category_2))
sl2_bar.add_yaxis(
    series_name="前80%销量",
    y_axis=list(sales_lei2m80.buy_mount),
    yaxis_index=0,
    label_opts=opts.LabelOpts(is_show=False)
)
sl2_bar.extend_axis(
    yaxis=opts.AxisOpts(
        type_="value",
        name="销量占比",
        min_=0,
        max_=1,
        position="right",
    )
)
sl2_bar.set_global_opts(
    tooltip_opts=opts.TooltipOpts(is_show=True,trigger='axis',axis_pointer_type='cross'),
    xaxis_opts=opts.AxisOpts(type_='category'),
    yaxis_opts=opts.AxisOpts(type_='value',min_=0, max_=1200, position="left"),
    title_opts=opts.TitleOpts(title="二级类目商品销量以及贡献前80%销量的情况"),
    legend_opts=opts.LegendOpts(pos_top="18")
)

sl2_line=Line()
sl2_line.add_xaxis(xaxis_data=list(sales_lei2m80.category_2))
sl2_line.add_yaxis(
    series_name="销量占比",
    y_axis=list(sales_lei2m80.cs_percent),
    yaxis_index=1,
    label_opts=opts.LabelOpts(is_show=False)
)

sl2_bar.overlap(sl2_line)
grid2=Grid()
grid2.add(
    sl2_bar, 
    opts.GridOpts(pos_left="5%", pos_right="5%"),
    is_control_axis_index=True
)
grid2.render_notebook()

['50013636', '50018831', '211122', '50010558', '50006602', '50011993', '50013207', '50016006', '250822', '50012359', '50016010', '121394024', '50010555', '50002524', '50012474', '50006520', '50003700', '50013187', '50023591', '50010566', '50023790', '50023670', '121452056', '251725', '50010557', '50007011', '121462075', '50012564', '50012442', '50010549', '122616024', '50012365', '50016468', '50012364', '121466023', '50016036', '50013711', '50010544', '50018615', '50012375', '50012451', '50012361', '50010536', '50016116', '50009540', '50017100', '50023536', '121408024', '50012363', '50009547', '50000820', '50016704', '50023531', '121540027', '50008847', '50024842', '50023807', '50010545', '50012549', '50006843', '50016454', '121382040', '252819', '50024439', '50008807', '50012788', '50012777', '50010542', '50012424', '121404037', '121424027', '50012360', '50023663', '50016240', '50018436', '50016706', '50012244', '121434042', '50228021', '50018825', '50015727', '50014744', '50012462', 

可重点关注贡献前80%销量的商品一、二级类目对应的母婴商品的营销。

In [63]:
#2014年
#母婴购物销量为1的商品二级类目分析
print("2014年母婴商品二级类目数：",buy14['category_2'].nunique())
sales_lei2min=buy14.groupby('category_2')['buy_mount'].sum().sort_values().reset_index()

sales_lei2min=sales_lei2min[sales_lei2min.buy_mount==1]

sales_lei2minm=sales_lei2min['category_2'].count()
print("母婴购物销量为1的商品二级类目数：",sales_lei2minm)
print("母婴购物销量为1的商品二级类目数占比：",round(sales_lei2minm/sales_lei2z,2))
print(list(sales_lei2min.category_2))

2014年母婴商品二级类目数： 576
母婴购物销量为1的商品二级类目数： 86
母婴购物销量为1的商品二级类目数占比： 0.15
['50250020', '50006031', '50015913', '50005988', '50023633', '50023613', '50026464', '50005915', '50005790', '50002682', '50026468', '50016039', '50001707', '50000833', '251220', '50026490', '122696024', '122616025', '121484020', '121480041', '121476021', '121474051', '121474037', '121470050', '121462037', '121456032', '121456020', '50136026', '50006040', '50015399', '50026458', '50023642', '50012479', '50012486', '50023695', '50012433', '50023696', '50012396', '50012390', '50012381', '50023710', '50023717', '50012836', '50011838', '50013194', '121452029', '50023798', '50013197', '50023648', '50013218', '50023647', '50013985', '50024169', '50008709', '50008707', '50024184', '50024247', '50024621', '50014862', '50015071', '50023886', '121452027', '50005980', '50012477', '121426024', '50150022', '121424024', '50236022', '121390035', '121396023', '121422041', '50158022', '121400029', '50018704', '50016902', '50152021', '121

对总销量为1的二级类目对应的商品，查看是否为耐用品。

In [32]:
sales_lei1n2=buy14.groupby('category_1')['category_2'].nunique().reset_index()

cat1_order=list(sales_lei1.category_1)
sales_lei1n2['category_1']=pd.Categorical(sales_lei1n2['category_1'], categories=cat1_order, ordered=True)
sales_lei1n2=sales_lei1n2.sort_values(by='category_1')

l1n2=Bar()
l1n2.add_xaxis(xaxis_data=list(sales_lei1n2.category_1))
l1n2.add_yaxis(
    series_name="类目数",
    y_axis=list(sales_lei1n2.category_2)
)
l1n2.set_global_opts(
    tooltip_opts=opts.TooltipOpts(is_show=True,trigger='axis',axis_pointer_type='cross'),
    xaxis_opts=opts.AxisOpts(type_='category'),
    yaxis_opts=opts.AxisOpts(type_='value'),
    title_opts=opts.TitleOpts(title="商品一级类目的二级类目数")
)
l1n2.render_notebook()

In [33]:
sales_lns=buy14.groupby('category_1').agg(lein=('category_2','nunique'),salesn=('buy_mount','sum')).reset_index()

sales_lns['category_1']=pd.Categorical(sales_lns['category_1'], categories=cat1_order, ordered=True)
sales_lns=sales_lns.sort_values(by='category_1')

slns=Bar()
slns.add_xaxis(xaxis_data=list(sales_lns.category_1))
slns.add_yaxis(
    series_name="类目数",
    y_axis=list(round(sales_lns.salesn/sales_lns.lein,2))
)
slns.set_global_opts(
    tooltip_opts=opts.TooltipOpts(is_show=True,trigger='axis',axis_pointer_type='cross'),
    xaxis_opts=opts.AxisOpts(type_='category'),
    yaxis_opts=opts.AxisOpts(type_='value'),
    title_opts=opts.TitleOpts(title="商品一级类目的平均销量")
)
slns.render_notebook()

由图可以看出，50008168、122650008、38一级类目商品的二级类目的平均销量较高，且对应的二级类目数较少，
可以把这两类商品和销量少的商品捆绑消费，以带动消费。

In [34]:
#数据导入
datababy=pd.read_csv('E:/数据分析师相关/数据分析项目相关/淘宝母婴购物行为数据集/tianchi_mum_baby.csv')

In [35]:
#数据查看
datababy.info()
datababy.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   user_id   953 non-null    int64
 1   birthday  953 non-null    int64
 2   gender    953 non-null    int64
dtypes: int64(3)
memory usage: 22.5 KB


user_id     0
birthday    0
gender      0
dtype: int64

In [36]:
#数据处理
datababy['user_id']=datababy['user_id'].astype(str)
datababy['birthday']=datababy['birthday'].astype(str)
datababy['birthday']=pd.to_datetime(datababy['birthday'])
datababy['gender']=datababy['gender'].astype(str)

In [37]:
datababy['gender'].value_counts()

0    489
1    438
2     26
Name: gender, dtype: int64

In [38]:
datababy=datababy[datababy['gender'].isin(['0','1'])]
datababy['gender']=datababy['gender'].replace({'0': 'boy', '1': 'girl'})

In [39]:
datababy.duplicated().sum()

0

In [40]:
datababy.head(5)

,user_id,birthday,gender
0,2757,2013-03-11,girl
1,415971,2012-11-11,boy
2,1372572,2012-01-30,girl
3,10339332,2011-09-10,boy
4,10642245,2013-02-13,boy


In [41]:
datababy.birthday.describe()

count                     927
unique                    793
top       2013-03-09 00:00:00
freq                        4
first     1984-06-16 00:00:00
last      2015-08-15 00:00:00
Name: birthday, dtype: object

In [42]:
datababy=datababy[datababy['birthday']>'2002-07-10']

In [43]:
datababy.birthday.describe()

count                     926
unique                    792
top       2013-03-09 00:00:00
freq                        4
first     2002-12-05 00:00:00
last      2015-08-15 00:00:00
Name: birthday, dtype: object

In [44]:
print("用户数：",datababy.user_id.nunique())
print("婴幼儿人数：",datababy.user_id.count())

用户数： 926
婴幼儿人数： 926


In [45]:
databb=pd.merge(databuy,datababy,on='user_id',how='inner')

In [46]:
databb.head(5)

,user_id,auction_id,category_2,category_1,buy_mount,day,year,quarter,month,days,birthday,gender
0,124613323,15186722756,50006095,28,1,2012-07-02,2012,3,07,02,2011-04-20,girl
1,797916383,16261260713,50013636,50008168,5,2012-07-02,2012,3,07,02,2013-04-18,boy
2,460971987,10082122933,50013881,28,1,2012-07-05,2012,3,07,05,2011-10-01,boy
3,38060393,14027224169,50018436,50014815,1,2012-07-07,2012,3,07,07,2011-08-01,girl
4,517793335,16719576868,50006843,38,2,2012-07-10,2012,3,07,10,2011-07-10,girl


In [47]:
databb['age']=round((databb['day']-databb['birthday'])/pd.Timedelta('365 days'),1)

In [48]:
databb.age.describe()

count    929.000000
mean       1.802906
std        2.001212
min       -2.100000
25%        0.300000
50%        1.200000
75%        2.800000
max       11.800000
Name: age, dtype: float64

In [49]:
databb=databb[(databb['age']<7)&(databb['age']>0)]

In [50]:
databb.age.describe()

count    760.000000
mean       2.021842
std        1.593985
min        0.100000
25%        0.800000
50%        1.600000
75%        3.000000
max        6.900000
Name: age, dtype: float64

In [51]:
#性别分析
baby_gb=databb.groupby('gender')['user_id'].nunique().reset_index()

baby_gn=baby_gb[baby_gb['gender']=="girl"].iloc[0,1]
baby_bn=baby_gb[baby_gb['gender']=="boy"].iloc[0,1]

print("婴幼儿总人数：", baby_gn+baby_bn)
print("女婴幼儿人数：",baby_gn)
print("男婴幼儿人数：",baby_bn)

gbn=Pie()
gbn.add("", [list(z) for z in zip(list(baby_gb.gender), list(baby_gb.user_id))])
gbn.set_global_opts(title_opts=opts.TitleOpts(title="男女婴幼儿人数占比"))
gbn.set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))
gbn.render_notebook()

婴幼儿总人数： 758
女婴幼儿人数： 356
男婴幼儿人数： 402


In [52]:
leig=databb[databb['gender']=='girl'].groupby('category_1')['buy_mount'].sum().reset_index().rename(columns = {'buy_mount':'g_lnum'})
leib=databb[databb['gender']=='boy'].groupby('category_1')['buy_mount'].sum().reset_index().rename(columns = {'buy_mount':'b_lnum'})

datalgb=pd.merge(leig,leib,on='category_1')
datalgb['gb_num']=datalgb['g_lnum']+datalgb['b_lnum']
datalgb['g_np']=round(100*datalgb['g_lnum']/datalgb['gb_num'],2)
datalgb['b_np']=round(100*datalgb['b_lnum']/datalgb['gb_num'],2)

lgb=Bar()
lgb.add_xaxis(list(datalgb.category_1))
lgb.add_yaxis("女", list(datalgb.g_np), stack="stack1")
lgb.add_yaxis("男", list(datalgb.b_np), stack="stack1")
lgb.set_series_opts(label_opts=opts.LabelOpts(formatter="{c}%"))
lgb.set_global_opts(
    title_opts=opts.TitleOpts(title="男女婴幼儿一级类目商品销量占比")
)
lgb.render_notebook()

男女婴幼儿人数较为均匀，由图可以看出，男女婴幼儿在38、50014815一级类目的商品销量上是呈现出明显差异的。
对于这两类商品，可以适当地针对性别进行营销。

In [53]:
glp=(
    Pie()
    .add(
    "女婴幼儿", 
    [list(z) for z in zip(list(datalgb.category_1), list(datalgb.g_lnum))]
)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="女婴幼儿各一级类目商品销量占比"), 
        legend_opts=opts.LegendOpts(pos_top="18")
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: \n {d}%"))
)
glp.render_notebook()

In [54]:
blp=(
    Pie()
    .add(
    "男婴幼儿", 
    [list(z) for z in zip(list(datalgb.category_1), list(datalgb.b_lnum))]
)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="男婴幼儿各大类商品销量占比"), 
        legend_opts=opts.LegendOpts(pos_top="17"),
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: \n {d}%"))
)
blp.render_notebook()

对于男女婴幼儿，他们在50014815、50008168这两类商品上需求量有所差异。
女婴幼儿对50008168类商品需求量更大，男婴幼儿对50014815类商品需求量更大。
对于这两类商品可针对性别进行营销。

**总结**
1. 数据方面：
  2012年7月2日到2015年2月5日期间，淘宝母婴购物总用户数为29914，总订单数为29941，总购买量为49778，平均每个用户的购买量为1~2，平均每个订单的购买量为1~2。
  分析：总用户数很多，每用户购买量、每订单购买量少，用户黏性差，可能是因为数据量较少，也可能是因为商品不满足客户需求。

  婴幼儿共758名，女婴幼儿共356名，占总人数的46.97%；男婴幼儿共402名，占总人数的53.03%。
男女婴幼儿人数较均匀。

2. 流量分析：
  按年来看，从2013年到2014年，母婴商品销量是明显增加的，预计2015年母婴商品销量也会递增。
    
  按季度、月、日来看，每年第一季度销量最少，因为春节期间消费低迷，促销活动较少；第四季度销量最多，主要因为双十一活动。
此外，每年的5月销量较高，因为有母亲节活动、520活动，以及其他不定期的品牌活动。
另外，6、12月紧跟着销量高的5、11月，虽然有618活动、双十二活动，但销量不高，可能因为用户在5、11月囤够货，需求减少，也可能是因为活动力度不大。
  建议：应该在第一季度适当开展促销活动，比如在春节前一个月这段时间。
活动力度大的促销活动之间最好间隔2、3个月进行，期间可以进行活动力度较小的促销活动，以更好地刺激消费。

3. 类别分析：
  母婴商品类别包括一、二级类目，一级类目数有6个，每年的二级类目数不同，2013年有444个二级类目，2014年有576个二级类目，主要分析2014年数据。
对于2014年，贡献前80%母婴购物销量的商品一级类目主要为'28'、'50008168'类，占总一级类目数的33%；
贡献前80%销量的商品二级类目主要为'50013636'、'50018831'、'211122'、'50010558'、'50006602'类，有116个，占总二级类目数的20%。
销量为1的商品二级类目数为86，占总二级类目数的15%。
  建议：对于贡献前80%销量的类目对应的商品可多加关注，加大营销力度，可适当把销量高的商品与销量低的商品捆绑售卖；
对于销量为1的类目对应的商品，查看是否为耐用必需品，若不是，则打折促销，减少、清空库存，下架商品。

  在6个商品一级类目中，'50008168'、'122650008'、'38'类目的子类平均销量较高，对应的子类数量较少。说明这些类目对应的商品需求大、市场稳定。
  建议：可进一步研究购买这些类目商品的客户需求，以改进商品、研发新产品，更好地满足客户需求。

4. 性别分析：
  男女婴幼儿在'38'、'50014815'一级类目母婴商品销量上呈现出明显的性别差异，男婴幼儿在这两类商品的销量占比多，为70~80%。
对于女婴幼儿，在一级类目商品中，'50008168'类商品销量占比最多，为40.92%，'50014815'类商品销量占比较多，为19.77%；
而对于男婴幼儿，在一级类目商品中，'50014815'类商品销量占比最多，为39.77%，'50008168'类商品销量占比较多，为25.13%。
  建议：男女婴幼儿对'38'类商品需求量差别不大，但在销量上差别明显，可以适当挖掘女婴幼儿对该类商品的需求，改进商品。
男女婴幼儿对'50008168'类商品需求量差别较大，但在销量上差别不大，可以优化该类商品的产品定位，调整营销策略。
